<B>TCC PUC MINAS - MAIO/2022</B><BR>
ANÁLISE DE PARCELAMENTO DE DÉBITOS INSCRITOS EM DÍVIDA ATIVA DA UNIÃO ATRAVÉS DE MACHINE LEARNING<BR>
FELIPE MARTINS<BR>

06 - FUSÃO (MERGE) DOS DATASETS DEBITOS E CNPJ
    
Tempo médio de processamento deste notebook: 90 minutos.

In [1]:
import pandas as pd

## Importadando dataset da base CNPJ

In [2]:
%%time
# IMPORTANDO ARQUIVO 04_cnpj_final.csv
df_cnpj_final = pd.read_csv("04_cnpj_final.csv", dtype={"CNPJ_BASICO":"category", "NAT_JURID":"category", "PORTE":"category"})  #nrows=5

Wall time: 6min 30s


In [3]:
df_cnpj_final.head()

,Unnamed: 0,CNPJ_BASICO,SIT_CADAST,CNAE_PRINC,NAT_JURID,PORTE,ATIV_>10_ANOS
0,0,44.628.188/0001-92,2,85,2062,03,S
1,1,07.349.468/0001-35,2,56,2135,01,S
2,2,25.405.580/0001-28,4,47,2135,01,S
3,3,62.534.730/0001-20,2,47,2062,01,S
4,4,80.969.025/0001-37,8,23,2135,01,S


In [4]:
df_cnpj_final = df_cnpj_final.drop(columns=['Unnamed: 0'])

In [5]:
# RENOMEANDO COLUNA (APESAR DE O DATASET CNPJ NÃO TER CPF, É NECESSÁRIO O MESMO NOME DE COLUNA PARA O MERGE)
df_cnpj_final = df_cnpj_final.rename(columns = {'CNPJ_BASICO': 'CPF_CNPJ'})

In [6]:
df_cnpj_final.head()

,CPF_CNPJ,SIT_CADAST,CNAE_PRINC,NAT_JURID,PORTE,ATIV_>10_ANOS
0,44.628.188/0001-92,2,85,2062,03,S
1,07.349.468/0001-35,2,56,2135,01,S
2,25.405.580/0001-28,4,47,2135,01,S
3,62.534.730/0001-20,2,47,2062,01,S
4,80.969.025/0001-37,8,23,2135,01,S


In [7]:
df_cnpj_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51528473 entries, 0 to 51528472
Data columns (total 6 columns):
 #   Column         Dtype   
---  ------         -----   
 0   CPF_CNPJ       category
 1   SIT_CADAST     int64   
 2   CNAE_PRINC     int64   
 3   NAT_JURID      category
 4   PORTE          category
 5   ATIV_>10_ANOS  object  
dtypes: category(3), int64(2), object(1)
memory usage: 2.8+ GB


## Importadando dataset de débitos

In [8]:
%%time
# IMPORTANDO ARQUIVO 05_debitos.csv
df_debitos = pd.read_csv("05_debitos.csv")   #nrows=5

Wall time: 23.6 s


In [9]:
df_debitos = df_debitos.drop(columns=['Unnamed: 0'])

In [10]:
df_debitos.head()

,CPF_CNPJ,TIPO_PESSOA,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,INSC_>1_ANO,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO
0,11.859.590/0001-29,PJ,2141900331458,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,NAO,55464.40,S,S,6.0,N
1,05.574.789/0001-08,PJ,2151000016030,Em cobrança,ATIVA AJUIZADA,Multa - CLT,SIM,55446.40,S,S,6.0,N
2,XXX856.122XX,PF,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,OUTROS,NAO,3182.70,N,S,2.0,S
3,14.177.984/0001-95,PJ,2160000092445,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,3457828.22,S,S,9.0,N
4,02.793.722/0001-76,PJ,2161900910100,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,NAO,1521.30,S,N,2.0,N


In [11]:
df_debitos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14992922 entries, 0 to 14992921
Data columns (total 12 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CPF_CNPJ                 object 
 1   TIPO_PESSOA              object 
 2   NUMERO_INSCRICAO         int64  
 3   TIPO_SITUACAO_INSCRICAO  object 
 4   SITUACAO_INSCRICAO       object 
 5   RECEITA_PRINCIPAL        object 
 6   INDICADOR_AJUIZADO       object 
 7   VALOR_CONSOLIDADO        float64
 8   INSC_>1_ANO              object 
 9   HA_OUTROS_DEBITOS        object 
 10  VALOR_FAIXA              float64
 11  PARCELADO                object 
dtypes: float64(2), int64(1), object(9)
memory usage: 1.3+ GB


## Realizando-se a fusão (merge) dos datasets

In [12]:
%%time
# FUSÃO DOS DATASET DÉBITOS E CNPJ_FINAL
# PROCESSAMENTO DEMORADO. POSSÍVEL PROBLEMA DE FALTA DE MEMÓRIA. ENCERRAR OUTROS PROGRAMAS E NOTEBOOKS ABERTOS.
df_debitos_cnpj = pd.merge(df_debitos, df_cnpj_final, how = 'left', on = 'CPF_CNPJ')

## Procedimentos finais

In [13]:
df_debitos_cnpj.head(100)

,CPF_CNPJ,TIPO_PESSOA,NUMERO_INSCRICAO,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,INSC_>1_ANO,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO,SIT_CADAST,CNAE_PRINC,NAT_JURID,PORTE,ATIV_>10_ANOS
0,11.859.590/0001-29,PJ,2141900331458,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,NAO,55464.40,S,S,6.0,N,4.0,47.0,2062,03,S
1,05.574.789/0001-08,PJ,2151000016030,Em cobrança,ATIVA AJUIZADA,Multa - CLT,SIM,55446.40,S,S,6.0,N,4.0,46.0,2062,05,S
2,XXX856.122XX,PF,2142000534105,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,OUTROS,NAO,3182.70,N,S,2.0,S,NaN,NaN,NaN,NaN,NaN
3,14.177.984/0001-95,PJ,2160000092445,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,3457828.22,S,S,9.0,N,8.0,26.0,2054,05,S
4,02.793.722/0001-76,PJ,2161900910100,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,NAO,1521.30,S,N,2.0,N,4.0,41.0,2062,01,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,15.070.453/0001-61,PJ,2161500682703,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,9853.33,S,S,3.0,N,8.0,43.0,2135,01,N
96,21.456.849/0001-44,PJ,2162100036411,Em cobrança,ATIVA EM COBRANCA,COFINS,NAO,16976.04,N,S,4.0,N,2.0,86.0,2240,05,N
97,06.314.820/0001-34,PJ,2121100100074,Em cobrança,ATIVA AJUIZADA,IRPJ,SIM,3551.05,S,S,2.0,N,4.0,82.0,2135,01,S
98,XXX520.602XX,PF,2111900981108,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,IRPF,NAO,8769.87,S,S,3.0,S,NaN,NaN,NaN,NaN,NaN


In [14]:
df_debitos_cnpj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14992922 entries, 0 to 14992921
Data columns (total 17 columns):
 #   Column                   Dtype   
---  ------                   -----   
 0   CPF_CNPJ                 object  
 1   TIPO_PESSOA              object  
 2   NUMERO_INSCRICAO         int64   
 3   TIPO_SITUACAO_INSCRICAO  object  
 4   SITUACAO_INSCRICAO       object  
 5   RECEITA_PRINCIPAL        object  
 6   INDICADOR_AJUIZADO       object  
 7   VALOR_CONSOLIDADO        float64 
 8   INSC_>1_ANO              object  
 9   HA_OUTROS_DEBITOS        object  
 10  VALOR_FAIXA              float64 
 11  PARCELADO                object  
 12  SIT_CADAST               float64 
 13  CNAE_PRINC               float64 
 14  NAT_JURID                category
 15  PORTE                    category
 16  ATIV_>10_ANOS            object  
dtypes: category(2), float64(4), int64(1), object(10)
memory usage: 1.8+ GB


In [15]:
# AJUSTANDO TIPO DE DADOS RESTANTES PARA OBJECT/STRING
df_debitos_cnpj["NUMERO_INSCRICAO"] = df_debitos_cnpj["NUMERO_INSCRICAO"].astype("object")
df_debitos_cnpj["SIT_CADAST"] = df_debitos_cnpj["SIT_CADAST"].astype("object")
df_debitos_cnpj["CNAE_PRINC"] = df_debitos_cnpj["CNAE_PRINC"].astype("object")
df_debitos_cnpj["NAT_JURID"] = df_debitos_cnpj["NAT_JURID"].astype("object")
df_debitos_cnpj["PORTE"] = df_debitos_cnpj["PORTE"].astype("object")

In [16]:
df_debitos_cnpj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14992922 entries, 0 to 14992921
Data columns (total 17 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   CPF_CNPJ                 object 
 1   TIPO_PESSOA              object 
 2   NUMERO_INSCRICAO         object 
 3   TIPO_SITUACAO_INSCRICAO  object 
 4   SITUACAO_INSCRICAO       object 
 5   RECEITA_PRINCIPAL        object 
 6   INDICADOR_AJUIZADO       object 
 7   VALOR_CONSOLIDADO        float64
 8   INSC_>1_ANO              object 
 9   HA_OUTROS_DEBITOS        object 
 10  VALOR_FAIXA              float64
 11  PARCELADO                object 
 12  SIT_CADAST               object 
 13  CNAE_PRINC               object 
 14  NAT_JURID                object 
 15  PORTE                    object 
 16  ATIV_>10_ANOS            object 
dtypes: float64(2), object(15)
memory usage: 2.0+ GB


In [17]:
%%time
# EXPORTANTO DATASET FINAL (antes da exclusão das colunas desnecessárias para ML)
# Essa linha de comando nao consta no TCC. Foram utilizados apenas para uso interno.
df_debitos_cnpj.to_csv("06_dataset_final_antes_limpeza.csv")

Wall time: 1min 38s


In [18]:
# EXCLUINDO COLUNAS DESNECESSÁRIAS (LIMPEZA FINAL)
df_debitos_cnpj = df_debitos_cnpj.drop(columns=['CPF_CNPJ','NUMERO_INSCRICAO'])

In [19]:
df_debitos_cnpj.head()

,TIPO_PESSOA,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,INSC_>1_ANO,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO,SIT_CADAST,CNAE_PRINC,NAT_JURID,PORTE,ATIV_>10_ANOS
0,PJ,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,NAO,55464.40,S,S,6.0,N,4.0,47.0,2062,03,S
1,PJ,Em cobrança,ATIVA AJUIZADA,Multa - CLT,SIM,55446.40,S,S,6.0,N,4.0,46.0,2062,05,S
2,PF,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,OUTROS,NAO,3182.70,N,S,2.0,S,NaN,NaN,NaN,NaN,NaN
3,PJ,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,3457828.22,S,S,9.0,N,8.0,26.0,2054,05,S
4,PJ,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,NAO,1521.30,S,N,2.0,N,4.0,41.0,2062,01,S


In [20]:
df_debitos_cnpj.tail()

,TIPO_PESSOA,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,INDICADOR_AJUIZADO,VALOR_CONSOLIDADO,INSC_>1_ANO,HA_OUTROS_DEBITOS,VALOR_FAIXA,PARCELADO,SIT_CADAST,CNAE_PRINC,NAT_JURID,PORTE,ATIV_>10_ANOS
14992917,PJ,Benefício Fiscal,ATIVA AJUIZADA NEGOCIADA NO SISPAR,Multa - CLT,SIM,9487.12,S,S,3.0,S,4.0,86.0,2062,01,S
14992918,PJ,Em cobrança,ATIVA EM COBRANCA,SIMP NAC - MEI,NAO,5173.43,N,N,3.0,N,4.0,14.0,2135,01,N
14992919,PJ,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,109317.16,S,S,7.0,N,2.0,41.0,2062,03,S
14992920,PF,Em cobrança,ATIVA EM COBRANCA,IRPF,NAO,2372.02,N,S,2.0,N,NaN,NaN,NaN,NaN,NaN
14992921,PJ,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,NAO,4997.24,N,N,2.0,N,2.0,47.0,2135,01,N


In [21]:
df_debitos_cnpj.nunique()

TIPO_PESSOA                      2
TIPO_SITUACAO_INSCRICAO          5
SITUACAO_INSCRICAO             117
RECEITA_PRINCIPAL               13
INDICADOR_AJUIZADO               2
VALOR_CONSOLIDADO          5496820
INSC_>1_ANO                      2
HA_OUTROS_DEBITOS                2
VALOR_FAIXA                     10
PARCELADO                        2
SIT_CADAST                       5
CNAE_PRINC                      80
NAT_JURID                       72
PORTE                            3
ATIV_>10_ANOS                    2
dtype: int64

In [22]:
# REORDENANDO COLUNAS (ATRIBUTO-CLASSE (ALVO) (PARCELADO) DEVE SER A ÚLTIMA)
df_debitos_cnpj = df_debitos_cnpj[['TIPO_PESSOA','TIPO_SITUACAO_INSCRICAO','SITUACAO_INSCRICAO','RECEITA_PRINCIPAL','INDICADOR_AJUIZADO','HA_OUTROS_DEBITOS','INSC_>1_ANO','VALOR_CONSOLIDADO','VALOR_FAIXA','SIT_CADAST','CNAE_PRINC','NAT_JURID','PORTE','ATIV_>10_ANOS','PARCELADO']]

In [23]:
df_debitos_cnpj.head(15)

,TIPO_PESSOA,TIPO_SITUACAO_INSCRICAO,SITUACAO_INSCRICAO,RECEITA_PRINCIPAL,INDICADOR_AJUIZADO,HA_OUTROS_DEBITOS,INSC_>1_ANO,VALOR_CONSOLIDADO,VALOR_FAIXA,SIT_CADAST,CNAE_PRINC,NAT_JURID,PORTE,ATIV_>10_ANOS,PARCELADO
0,PJ,Em cobrança,ATIVA EM COBRANCA,SIMPLES NACIONAL,NAO,S,S,55464.40,6.0,4.0,47.0,2062,03,S,N
1,PJ,Em cobrança,ATIVA AJUIZADA,Multa - CLT,SIM,S,S,55446.40,6.0,4.0,46.0,2062,05,S,N
2,PF,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,OUTROS,NAO,S,N,3182.70,2.0,NaN,NaN,NaN,NaN,NaN,S
3,PJ,Em cobrança,ATIVA AJUIZADA,COFINS,SIM,S,S,3457828.22,9.0,8.0,26.0,2054,05,S,N
4,PJ,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,NAO,N,S,1521.30,2.0,4.0,41.0,2062,01,S,N
5,PJ,Em cobrança,ATIVA A SER AJUIZADA,CSLL,NAO,S,S,5789.88,3.0,2.0,43.0,2135,01,S,N
6,PF,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,IRPF,NAO,N,S,6413.86,3.0,NaN,NaN,NaN,NaN,NaN,S
7,PJ,Em cobrança,ATIVA EM COBRANCA,Multa Isolada,NAO,N,S,3005.83,2.0,8.0,47.0,2135,01,N,N
8,PF,Em cobrança,ATIVA AJUIZADA,IRPF,SIM,S,S,7920.98,3.0,NaN,NaN,NaN,NaN,NaN,N
9,PJ,Benefício Fiscal,ATIVA NAO AJUIZAVEL NEGOCIADA NO SISPAR,PIS,NAO,S,S,247882.76,7.0,2.0,28.0,2062,05,N,S


In [24]:
%%time
# EXPORTANTO DATASET FINAL SEM CPF/CNPJ, NUMERO_INSCRICAO
# TODOS OS DADOS AINDA SÃO CATEGÓRICOS. SERÃO NECESSÁRIOS EM 08_analise_exploratoria.ypynb
# PARA MACHINE LEARNING, SERÁ NECESSÁRIA A CONVERSÃO PARA DADOS CONTÍNUOS. SERÁ FEITA EM 07_conv_dados_continuos.ipynb.
df_debitos_cnpj.to_csv("06_dataset_final_com_dados_categoricos.csv")

Wall time: 1min 24s


In [25]:
df_debitos_cnpj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14992922 entries, 0 to 14992921
Data columns (total 15 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   TIPO_PESSOA              object 
 1   TIPO_SITUACAO_INSCRICAO  object 
 2   SITUACAO_INSCRICAO       object 
 3   RECEITA_PRINCIPAL        object 
 4   INDICADOR_AJUIZADO       object 
 5   HA_OUTROS_DEBITOS        object 
 6   INSC_>1_ANO              object 
 7   VALOR_CONSOLIDADO        float64
 8   VALOR_FAIXA              float64
 9   SIT_CADAST               object 
 10  CNAE_PRINC               object 
 11  NAT_JURID                object 
 12  PORTE                    object 
 13  ATIV_>10_ANOS            object 
 14  PARCELADO                object 
dtypes: float64(2), object(13)
memory usage: 1.8+ GB
